In [2]:
import sys
from pathlib import Path

# Ubicación del notebook
NOTEBOOK_DIR = Path.cwd()

# Raíz del proyecto = subir un nivel desde Notebooks/
PROJECT_ROOT = NOTEBOOK_DIR.parent

# Añadir raíz del proyecto al sys.path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Proyecto raíz detectado:", PROJECT_ROOT)

Proyecto raíz detectado: c:\Proyectos\EduFinance\EduFinance_Simulator


In [8]:
import pandas as pd

from utils.paths import DATA_DIR
from utils.loader import load_csv, save_csv
from utils.cleaning import normalize_market_timeseries

print("📁 DATA_DIR:", DATA_DIR)
END_DATE   = "2025-09-05"


📁 DATA_DIR: C:\Proyectos\EduFinance\EduFinance_Simulator\data


In [10]:
#Cargar datos crudos
df_raw = load_csv(DATA_DIR / "raw_market_data.csv")

#Ejecutamos el pipeline de limpieza y normalización
market_df_clean,meta = normalize_market_timeseries(df_raw, END_DATE, strict=True)

#imprimir algunas filas para verificar
print(market_df_clean.head())

#Guardar los datos limpios
save_csv(market_df_clean, DATA_DIR / "clean_market_data.csv")

         date   ticker asset_class        close currency
0  2017-11-09  BTC-USD      Crypto  7143.580078      USD
1  2017-11-10  BTC-USD      Crypto  6618.140137      USD
2  2017-11-13  BTC-USD      Crypto  6559.490234      USD
3  2017-11-14  BTC-USD      Crypto  6635.750000      USD
4  2017-11-15  BTC-USD      Crypto  7315.540039      USD
Archivo guardado en: C:\Proyectos\EduFinance\EduFinance_Simulator\data\clean_market_data.csv


In [11]:
"""
Sanity check: Cobertura temporal por ticker

El objetivo de este bloque es verificar que la limpieza de datos
mediante cleaning.py fue correcta para cada activo.

1. Agrupamos por 'ticker'.
2. Calculamos:
  - first_date: primera fecha disponible en la serie.
  - last_date : última fecha disponible.
  - n_rows    : cantidad total de observaciones (filas).
  - n_missing : cantidad de valores nulos en 'close'.
  - mean      : promedio de los valores 'close'.
  - var       : varianza de los valores 'close'.  
  - std       : desviación estándar de los valores 'close'.
  - asset_class: clase de activo (ej. 'crypto', 'stock', etc.).
  - currency   : moneda en la que está denominado el activo.
3. Ordenamos los resultados por ticker para facilitar la inspección.

Este chequeo es importante porque:
  - Nos permite verificar que la limpieza de datos se realizó correctamente.
    - Identificamos posibles problemas o inconsistencias en los datos.
    - Nos da una visión general de la cobertura temporal y calidad de los datos
        para cada activo.

"""


summary_clean = (
    market_df_clean.groupby("ticker")
    .agg(
        first_date=("date", "min"),
        last_date=("date", "max"),
        n_rows=("close", "count"),
        n_missing=("close", lambda s: s.isna().sum()),
        mean=("close", "mean"),
        var=("close", "var"),
        std=("close", "std"),
        asset_class=("asset_class", "first"),
        currency=("currency", "first"),
    )
    .reset_index()
)

summary_clean.to_csv(DATA_DIR / "clean_market_summary.csv", index=False)
summary_clean.style.background_gradient(cmap="Blues")
display(summary_clean)

,ticker,first_date,last_date,n_rows,n_missing,mean,var,std,asset_class,currency
0,BTC-USD,2017-11-09,2025-09-04,1931,0,34100.146852,8.748030e+08,29577.069272,Crypto,USD
1,EUNL.DE,2017-11-09,2025-09-04,1931,0,68.826128,3.340429e+02,18.276842,ETF,EUR
2,QQQ,2017-11-09,2025-09-04,1931,0,307.956964,1.381476e+04,117.536217,ETF,USD
3,TSLA,2017-11-09,2025-09-04,1931,0,169.809921,1.384387e+04,117.659960,Stock,USD
4,V,2017-11-09,2025-09-04,1931,0,210.209250,3.825701e+03,61.852253,Stock,USD
5,VOO,2017-11-09,2025-09-04,1931,0,349.620733,1.105707e+04,105.152618,ETF,USD
6,XAR,2017-11-09,2025-09-04,1931,0,114.833072,9.342423e+02,30.565378,ETF,USD
7,XRP-USD,2017-11-09,2025-09-04,1931,0,0.720082,4.460942e-01,0.667903,Crypto,USD
